In [130]:
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import clear_output          
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier

## Importo el data_set_complete y el test100k

In [131]:
data_set_complete = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/data_set_complete.csv', parse_dates=['fechapostulacion', 'ultima_vista'], dtype={'sepostulo':'int8', 'idaviso':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'estudios':'category', 'estado':'category', 'edad':'int8', 'sexo':'category','nombre_zona':'category', 'tipo_de_trabajo':'category', 'nivel_laboral':'category', 'nombre_area':'category'})
data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13208286 entries, 0 to 13208285
Data columns (total 16 columns):
sepostulo               int8
idaviso                 int64
idpostulante            object
fechapostulacion        datetime64[ns]
ultima_vista            datetime64[ns]
veces_visto             int16
estudios                category
estado                  category
sexo                    category
edad                    int8
titulo                  object
nombre_zona             category
tipo_de_trabajo         category
nivel_laboral           category
nombre_area             category
denominacion_empresa    object
dtypes: category(7), datetime64[ns](2), int16(1), int64(1), int8(2), object(3)
memory usage: 755.8+ MB


In [132]:
test_100k = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/test_final_100k_complete.csv')
test_100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
id                      100000 non-null int64
idaviso                 100000 non-null int64
idpostulante            100000 non-null object
fechapostulacion        0 non-null float64
ultima_vista            34161 non-null object
veces_visto             100000 non-null int64
estudios                94388 non-null object
estado                  94388 non-null object
sexo                    100000 non-null object
edad                    100000 non-null int64
titulo                  100000 non-null object
nombre_zona             100000 non-null object
tipo_de_trabajo         100000 non-null object
nivel_laboral           95569 non-null object
nombre_area             100000 non-null object
denominacion_empresa    99988 non-null object
dtypes: float64(1), int64(4), object(11)
memory usage: 12.2+ MB


/home/rodrigo/ENV/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [133]:
data_set_complete.isnull().sum()

sepostulo                      0
idaviso                        0
idpostulante                   0
fechapostulacion         6604534
ultima_vista            11425840
veces_visto                    0
estudios                  204672
estado                    204672
sexo                           0
edad                           0
titulo                    831723
nombre_zona               831723
tipo_de_trabajo           831723
nivel_laboral            1126571
nombre_area               831723
denominacion_empresa      832327
dtype: int64

In [134]:
test_100k.isnull().sum()

id                           0
idaviso                      0
idpostulante                 0
fechapostulacion        100000
ultima_vista             65839
veces_visto                  0
estudios                  5612
estado                    5612
sexo                         0
edad                         0
titulo                       0
nombre_zona                  0
tipo_de_trabajo              0
nivel_laboral             4431
nombre_area                  0
denominacion_empresa        12
dtype: int64

## Analisis de la data

Creo que la fecha de postulacion no aporta mucho

In [135]:
data_set_complete = data_set_complete.drop(columns=['fechapostulacion','ultima_vista','veces_visto', 'denominacion_empresa','titulo','idaviso','idpostulante'])

In [136]:
data_set_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13208286 entries, 0 to 13208285
Data columns (total 9 columns):
sepostulo          int8
estudios           category
estado             category
sexo               category
edad               int8
nombre_zona        category
tipo_de_trabajo    category
nivel_laboral      category
nombre_area        category
dtypes: category(7), int8(2)
memory usage: 126.0 MB


In [137]:
test_100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
id                      100000 non-null int64
idaviso                 100000 non-null int64
idpostulante            100000 non-null object
fechapostulacion        0 non-null float64
ultima_vista            34161 non-null object
veces_visto             100000 non-null int64
estudios                94388 non-null object
estado                  94388 non-null object
sexo                    100000 non-null object
edad                    100000 non-null int64
titulo                  100000 non-null object
nombre_zona             100000 non-null object
tipo_de_trabajo         100000 non-null object
nivel_laboral           95569 non-null object
nombre_area             100000 non-null object
denominacion_empresa    99988 non-null object
dtypes: float64(1), int64(4), object(11)
memory usage: 12.2+ MB


In [138]:
muestra = data_set_complete.sample(1000000)

In [139]:
def df_encode_categorical_vars(df):
    for column in df.columns:
        if (df[column].dtype.name == 'category'):
            le = preprocessing.LabelEncoder()
            df[column] = le.fit_transform(df[column].astype(str))

In [140]:
def df_encode_object_vars(df):
    for column in df.columns:
        if (df[column].dtype == type(object)):
            le = preprocessing.LabelEncoder()
            df[column] = le.fit_transform(df[column].astype(str))

In [141]:
df_encode_categorical_vars(muestra)
df_encode_object_vars(test_100k)

In [142]:
muestra.head()

,sepostulo,estudios,estado,sexo,edad,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
4864640,1,5,2,0,24,1,1,4,45
13048353,0,4,2,0,38,5,1,4,179
418302,1,5,2,1,20,1,1,4,179
10618241,0,5,2,0,27,6,10,5,182
5855865,1,6,2,0,26,5,1,2,148


In [143]:
test_100k.head()

,id,idaviso,idpostulante,fechapostulacion,ultima_vista,veces_visto,estudios,estado,sexo,edad,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,0,739260,13661,NaN,20246,0,0,2,0,42,2909,1,1,1,29,336
1,1,739260,16942,NaN,20246,0,5,2,1,30,2909,1,1,1,29,336
2,2,739260,61148,NaN,20246,0,6,2,0,36,2909,1,1,1,29,336
3,3,758580,2178,NaN,20246,0,3,2,1,68,8535,1,1,3,164,2720
4,4,758580,24726,NaN,20246,0,6,1,0,32,8535,1,1,3,164,2720


In [144]:
muestra.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 4864640 to 9845444
Data columns (total 9 columns):
sepostulo          1000000 non-null int8
estudios           1000000 non-null int64
estado             1000000 non-null int64
sexo               1000000 non-null int64
edad               1000000 non-null int8
nombre_zona        1000000 non-null int64
tipo_de_trabajo    1000000 non-null int64
nivel_laboral      1000000 non-null int64
nombre_area        1000000 non-null int64
dtypes: int64(7), int8(2)
memory usage: 62.9 MB


## Grid search

In [145]:
train_set, test_set, train_classes, test_classes = train_test_split(muestra, muestra['sepostulo'], test_size=0.3)

In [146]:
parameters = {'min_samples_leaf':[1, 10, 100, 1000, 10000, 100000]}

randtree = RandomForestClassifier(n_jobs=4)
clf2 = GridSearchCV(randtree, parameters)
clf2.fit(train_set, train_classes)

print('Test:', clf2.score(test_set, test_classes), 'Entrenamiento:', clf2.score(train_set, train_classes))

Test: 1.0 Entrenamiento: 1.0


In [147]:
clf2.best_params_

{'min_samples_leaf': 1}

In [148]:
randtree = RandomForestClassifier(n_jobs=-1)
clf6 = RandomForestClassifier(n_estimators=20, min_samples_leaf=7, min_samples_split=4).fit(train_set, train_classes)
print('Test:', clf6.score(test_set, test_classes), 'Entrenamiento:', clf6.score(train_set, train_classes))

Test: 1.0 Entrenamiento: 1.0


In [151]:
test_100k['sepostulo'] = clf6.predict(test100k.drop(columns=['id', 'edad', 'idaviso','idpostulante', 'fechapostulacion','ultima_vista','titulo', 'denominacion_empresa']).astype(int))

NameError: name 'test100k' is not defined